# データサイエンス100本ノック（構造化データ加工編） - Python¶
## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\masay\anaconda3\envs\Python37 envs\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 演習問題
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [2]:
# df_receiptの最初の5件表示
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [3]:
#非会員をdf1として除外
df1 = df_receipt.query('not customer_id.str.startswith("Z")', engine='python')

In [4]:
#売上金額（amount）を顧客ID（customer_id）ごとに合計する df2
df2 = df1.groupby('customer_id').agg({'amount':'sum'}).reset_index()

In [5]:
#合計した売上金額を常用対数化（底=10）として顧客ID、売上金額合計とともに表示 結果は10件表示させれば良い。
df2['amount_log'] = np.log10(df2['amount'])
df2.head(10)

,customer_id,amount,amount_log
0,CS001113000004,1298,3.113275
1,CS001114000005,626,2.796574
2,CS001115000010,3044,3.483445
3,CS001205000004,1988,3.298416
4,CS001205000006,3337,3.523356
5,CS001211000025,456,2.658965
6,CS001212000027,448,2.651278
7,CS001212000031,296,2.471292
8,CS001212000046,228,2.357935
9,CS001212000070,456,2.658965


> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [6]:
#非会員をdf1として除外
df1 = df_receipt.query('not customer_id.str.startswith("Z")', engine='python')

In [7]:
#売上金額（amount）を顧客ID（customer_id）ごとに合計する df2
df2 = df1.groupby('customer_id').agg({'amount':'sum'}).reset_index()

In [8]:
#合計した売上金額を自然対数化(底=e）として顧客ID、売上金額合計とともに表示 結果は10件表示させれば良い。
df2['amount_log'] = np.log(df2['amount'])
df2.head(10)

,customer_id,amount,amount_log
0,CS001113000004,1298,7.168580
1,CS001114000005,626,6.439350
2,CS001115000010,3044,8.020928
3,CS001205000004,1988,7.594884
4,CS001205000006,3337,8.112827
5,CS001211000025,456,6.122493
6,CS001212000027,448,6.104793
7,CS001212000031,296,5.690359
8,CS001212000046,228,5.429346
9,CS001212000070,456,6.122493


> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [11]:
#df_productをdf3とする
df3 = df_product

In [13]:
df3.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [15]:
#各商品の利益額を単価（unit_price）から原価（unit_cost）の差額とし、利益（unit_profit）とする。
df3['unit_profit'] = df3['unit_price'] - df3['unit_cost']
#結果は10件表示
df3.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit
0,P040101001,4,401,40101,198.0,149.0,49.0
1,P040101002,4,401,40101,218.0,164.0,54.0
2,P040101003,4,401,40101,230.0,173.0,57.0
3,P040101004,4,401,40101,248.0,186.0,62.0
4,P040101005,4,401,40101,268.0,201.0,67.0
5,P040101006,4,401,40101,298.0,224.0,74.0
6,P040101007,4,401,40101,338.0,254.0,84.0
7,P040101008,4,401,40101,420.0,315.0,105.0
8,P040101009,4,401,40101,498.0,374.0,124.0
9,P040101010,4,401,40101,580.0,435.0,145.0


> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。 ただし、単価と原価にはNULLが存在することに注意せよ。

In [16]:
#df_productをdf4とする
df4 = df_product

In [17]:
#各商品の利益率を単価（unit_price）から原価（unit_cost）の差額としそれを単価（unit_price）で割って、利益率（unit_profit_rate）とする。
df4['unit_profit_rate'] = (df4['unit_price'] - df4['unit_cost']) / df4['unit_price']

In [18]:
#meanのパラメータskipnaを使用する。
#NaN 値を除外する（skipna=True）
df4['unit_profit_rate'].mean(skipna=True)

0.24911389885177

> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [19]:
#df_productをdf5とする
df5 = df_product

In [20]:
#pandas.Series.applyメソッドはlambdaと組み合わせると、applyメソッドへ引数を渡すことができるようになる。
#切り捨て np.floor()
#apply(lambda x: np.floor(x / 0.7)でx/0.7の切り捨て数値を原価（unit_cost）にapplyし、新たな単価（new_price）とする。
df5['new_price'] = df5['unit_cost'].apply(lambda x: np.floor(x / 0.7))

In [21]:
#新しい利益率（new_profit_rate）は、新たな単価（new_price）から原価（unit_cost）の差額を新たな単価（new_price）で割り算する。
df5['new_profit_rate'] = (df5['new_price'] - df5['unit_cost']) / df5['new_price']

In [23]:
df5.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit,unit_profit_rate,new_price,new_profit_rate
0,P040101001,4,401,40101,198.0,149.0,49.0,0.247475,212.0,0.297170
1,P040101002,4,401,40101,218.0,164.0,54.0,0.247706,234.0,0.299145
2,P040101003,4,401,40101,230.0,173.0,57.0,0.247826,247.0,0.299595
3,P040101004,4,401,40101,248.0,186.0,62.0,0.250000,265.0,0.298113
4,P040101005,4,401,40101,268.0,201.0,67.0,0.250000,287.0,0.299652
5,P040101006,4,401,40101,298.0,224.0,74.0,0.248322,320.0,0.300000
6,P040101007,4,401,40101,338.0,254.0,84.0,0.248521,362.0,0.298343
7,P040101008,4,401,40101,420.0,315.0,105.0,0.250000,450.0,0.300000
8,P040101009,4,401,40101,498.0,374.0,124.0,0.248996,534.0,0.299625
9,P040101010,4,401,40101,580.0,435.0,145.0,0.250000,621.0,0.299517


In [24]:
#math.floorはNaNでエラーとなるが、numpy.floorはエラーとならない